#deconvolution

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)

In [ ]:
object <- readRDS("./RDS/250520.combined_3.4_visium_label_update_hires.rds")

In [ ]:
object

In [ ]:
object <- subset(object, idents = c("Neutrophilic pustule", "Eccrine gland coil", "Adipocyte", "Eccrine gland duct"), invert = TRUE)


In [ ]:
ref <- readRDS("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/RDS/250416_integrated_total_rawcounts.rds")

In [ ]:
ref <- NormalizeData(ref)

In [ ]:
ref <- FindVariableFeatures(ref)

In [ ]:
Idents(ref) <- "label2"

In [ ]:
# the annotation is stored in the 'subclass' column of object metadata
DimPlot(ref, group.by = "label2", label = TRUE)

In [ ]:
ref@meta.data$label2 <- as.character(ref@meta.data$label2)
ref@meta.data$label2[ref@meta.data$label2 %in% c("BK-1", "BK-2", "BK-3")] <- "BK-1-3"
ref@meta.data$label2[ref@meta.data$label2 %in% c("SBK-1", "SBK-2")] <- "SBK-1-2"
ref@meta.data$label2[ref@meta.data$label2 %in% c("SBK-3", "SBK-4")] <- "SBK-3-4"

In [ ]:
ref <- subset(ref, idents = c("Melanocyte"), invert = TRUE)

In [ ]:
object@meta.data$label <- as.character(object@meta.data$label)

object@meta.data$label[object@meta.data$label %in% c("a/mDC", "pDC", "Mast cell")] <- "Myeloid cell"

In [ ]:
object

In [ ]:
anchors <- FindTransferAnchors(reference = ref, query = object, normalization.method="LogNormalize")
predictions.assay <- TransferData(anchorset = anchors, refdata = ref$label2, prediction.assay = TRUE,
    weight.reduction = object[["pca"]], dims = 1:30)
object[["predictions"]] <- predictions.assay

In [ ]:
data_matrix <- object@assays[["predictions"]]@data
cell_types <- object$label


In [ ]:
row_order <- c("BK-1-3", "SBK-1-2", "SBK-3-4", "SBK-5", "Lymphocyte", "Myeloid cell", "Fibroblast", "Endothelial", "Pericyte")

In [ ]:
col_order <- c('Basal KC','Differentiating KC','Terminally diff KC','Pustular KC','Lymphoid-CCL19+ niche','Myeloid cell', 'Inflammatory Fib/Mac','Fibroblast', 'Endo/Peri')

In [ ]:
data_matrix <- object@assays[["predictions"]]@data
data_matrix <- data_matrix[rownames(data_matrix) != "max", ]

cell_types <- object$label
cell_ids <- colnames(data_matrix)
cell_ids_by_type <- split(cell_ids, cell_types[cell_ids])

sum_values <- matrix(NA, nrow = nrow(data_matrix), ncol = length(unique(cell_types)))
colnames(sum_values) <- unique(cell_types)
for (cell_type in unique(cell_types)) {
  selected_columns <- cell_ids_by_type[[cell_type]]
  sum_values[, cell_type] <- rowSums(data_matrix[, selected_columns, drop = FALSE])
}
rownames(sum_values) <- rownames(data_matrix)

min_max_standarize <- function(x) {
  (x - min(x)) / (max(x) - min(x))
}

data <- t(apply(sum_values, 1, min_max_standarize))
data_df <- as.data.frame(data)
colnames(data_df) <- colnames(sum_values)
rownames(data_df) <- rownames(sum_values)

In [ ]:
rownames(sum_values) 
colnames(sum_values)  

In [ ]:
data_df <- data_df[row_order, col_order, drop=FALSE]

In [ ]:
library(pheatmap)
library(viridis)

In [ ]:
pheatmap(t(data_df), 
                   color = plasma(300), 
                   cluster_rows = FALSE, 
                   cluster_cols = FALSE,
                   show_rownames = TRUE,          
                   show_colnames = TRUE,
                   fontsize = 15,   
                   angle_col = "90")

In [ ]:
ht <- pheatmap(
  t(data_df),
  color         = plasma(300),
  cluster_rows  = FALSE,
  cluster_cols  = FALSE,
  show_rownames = TRUE,
  show_colnames = TRUE,
  fontsize  = 15,
  angle_col     = 90,     
  silent        = TRUE    
)

ggsave(
  filename = "./figure3/250422_correlation_heatmap_2.svg",
  plot     = ht,   
  device   = "svg",       
  width    = 5.5,           
  height   = 4
)

In [ ]:
object <- readRDS("./RDS/250520.combined_3.4_visium_label_update_hires.rds")

In [ ]:
object

In [ ]:
object <- subset(object, idents = c("Neutrophilic pustule", "Eccrine gland coil", "Adipocyte", "Eccrine gland duct"), invert = TRUE)


In [ ]:
ref@meta.data$label3 <- as.character(ref@meta.data$label3)
ref@meta.data$label3[ref@meta.data$label3 %in% c("BK-1", "BK-2", "BK-3")] <- "BK-1-3"
ref@meta.data$label3[ref@meta.data$label3 %in% c("SBK-1", "SBK-2")] <- "SBK-1-2"
ref@meta.data$label3[ref@meta.data$label3 %in% c("SBK-3", "SBK-4")] <- "SBK-3-4"

In [ ]:
ref@meta.data$label3[ref@meta.data$label3 %in% c("Superficial FIB", "FRC-like FIB", "Inf universal FIB", "Universal FIB")] <- "Fibroblast"

In [ ]:
anchors <- FindTransferAnchors(reference = ref, query = object, normalization.method="LogNormalize")
predictions.assay <- TransferData(anchorset = anchors, refdata = ref$label3, prediction.assay = TRUE,
    weight.reduction = object[["pca"]], dims = 1:30)
object[["predictions"]] <- predictions.assay

In [ ]:
row_order <- c("BK-1-3", "SBK-1-2", "SBK-3-4", "SBK-5", "Th17", "CD4em", "CD4n", "CD8/NK", "pDC", "mDC", "cDC2-2", "cDC2-1", "Macrophage", "Fibroblast", "Endothelial", "Pericyte")

In [ ]:
col_order <- c('Basal KC','Differentiating KC','Terminally diff KC','Pustular KC', 'Lymphoid-CCL19+ niche','pDC','a/mDC', 'Inflammatory Fib/Mac','Fibroblast', 'Endo/Peri')

In [ ]:
data_matrix <- object@assays[["predictions"]]@data
data_matrix <- data_matrix[rownames(data_matrix) != "max", ]

cell_types <- object$label
cell_ids <- colnames(data_matrix)
cell_ids_by_type <- split(cell_ids, cell_types[cell_ids])

sum_values <- matrix(NA, nrow = nrow(data_matrix), ncol = length(unique(cell_types)))
colnames(sum_values) <- unique(cell_types)
for (cell_type in unique(cell_types)) {
  selected_columns <- cell_ids_by_type[[cell_type]]
  sum_values[, cell_type] <- rowSums(data_matrix[, selected_columns, drop = FALSE])
}
rownames(sum_values) <- rownames(data_matrix)

min_max_standarize <- function(x) {
  (x - min(x)) / (max(x) - min(x))
}

data <- t(apply(sum_values, 1, min_max_standarize))
data_df <- as.data.frame(data)
colnames(data_df) <- colnames(sum_values)
rownames(data_df) <- rownames(sum_values)

In [ ]:
colnames(sum_values)

In [ ]:
rownames(sum_values)

In [ ]:
data_df <- data_df[row_order, col_order, drop=FALSE]

In [ ]:
pheatmap(t(data_df), 
                   color = plasma(300), 
                   cluster_rows = FALSE, 
                   cluster_cols = FALSE,
                   show_rownames = TRUE,          
                   show_colnames = TRUE,
                   fontsize = 15,   
                   angle_col = "90")